In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Latex, Markdown
import vaccines as lib
import sys
import locale
sign = lambda x: 1 if x >= 0 else -1

# Dados e configurações
plt.rcParams["figure.figsize"] = (10,6.66)
pd.set_option('display.max_rows', None)
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

data = pd.read_pickle('data.pkl.gz')
hospitalization = pd.read_pickle('hospitalization.pkl.gz')

In [26]:
display(Markdown("## Ferramenta para determinação do impacto da vacinação nas internações por COVID-19"))
default_plan = """95%"""
region_options = [(v, k) for k, v in data['name'].to_dict().items()]
vaccine_options = [('Coronavac/sinovac/butantan', (21, 20, 0.95)), ('Astrazeneca/Fiocruz', (90, 22, 0.95))]


@widgets.interact(
    region=widgets.Dropdown(options=region_options, description="UF"),
    vaccines=widgets.IntText(500_000, description="Doses"),
    rate=widgets.IntText(6_000, description="dose/dia"),
    vaccine=widgets.Dropdown(options=vaccine_options, description="Tipo vacina"),
    plan=widgets.Textarea(default_plan, description="Metas"),
)
def run(vaccines, region, rate, vaccine, plan, coarse=fixed(False)):
    global events, age_distribution, severe, result
    
    second_dose_delay, immunization_delay, eff = vaccine
    rates = lib.compute_schedule(rate, vaccines)
    
    if coarse:
        s = lib.population_80_plus(data.loc[region, "age_distribution"]).iloc[1:]
        age_distribution = pd.Series(s.values.reshape((2, len(s) // 2)).sum(0), index = s.index[1::2])
        age_distribution.name = region
    else:
        age_distribution = lib.population_80_plus(data.loc[region, "age_distribution"])

    plan = lib.parse_plan(plan, age_distribution)
    events = lib.execute_plan_safe(plan, rates, age_distribution, delay=second_dose_delay)
    severe = hospitalization.loc[region].iloc[::-1]
    duration = lib.by_periods(events['day'].max() + immunization_delay + second_dose_delay, 30)
    result = lib.VaccinationCampaign(events, duration=duration)
    
    pressure = result.damage_curve(severe, delay=immunization_delay, efficiency=eff)
    deaths = result.damage_curve(severe, delay=immunization_delay)
    display(Markdown(f"""
## Resultados
* **Total de doses:** {int(result.vaccines):n}
* **Pessoas vacinadas:** {int(result.vaccines // 2):n}
* **Dias de vacinação:** {result.campaign_duration}
* **Redução na hospitalização:** {100 - 100 * pressure.iloc[-1]:.1f}%
* **Redução dos óbitos:** {100 - 100 * deaths.iloc[-1]:.1f}%
    """))
    result.plot_hospitalization_pressure_curve(pressure, as_pressure=True)
    result.plot_vaccination_schedule()
    plt.show()
    
    
    
    display(Markdown(f"""
## Observações

O primeiro gráfico mostra a diminuição na pressão hospitalar esperada 
devido à proteção conferida pelas vacinas. A simulação considera que a 
taxa de infecção se mantêm constante, o que é uma suposição conservadora, 
especialmente em níveis mais altos de vacinação.
    """))


## Ferramenta para determinação do impacto da vacinação nas internações por COVID-19

interactive(children=(IntText(value=500000, description='Doses'), Dropdown(description='UF', options=(('Acre',…

In [27]:
for m in list(sys.modules):
    if m.startswith('vaccines'):
        del sys.modules[m]
import vaccines as lib

#campaign = lib.VaccinationCampaign(events, duration=120)#, age_distribution, severe)
#campaign.plot_hospitalization_pressure_curve(severe, delay=22)
# campaign.vaccines